In [ ]:
from py_ecc.bn128 import G1, G2, multiply, pairing, add, neg, eq, curve_order
from galois import Poly, GF
import random

In [ ]:
c.bn128 import G1, G2, multiply, pairing, add, neg, eq, curve_order
from galois import Poly, GF
import random

In [2]:
field = GF(curve_order) # Define the finite field based on the curve order

In [ ]:
class KZGCommitment:
    # Setup a KZG Commitment Scheme with a trusted setup phase.     
    def __init__(self, max_degree, field):
        self.max_degree = max_degree
        self.field = field
        self.srs = random.randint(1, curve_order - 1)
        self.srs_g1 = [multiply(G1, self.srs**i) for i in range(max_degree + 1)]
        self.srs_g2 = [G2, multiply(G2, self.srs)]
        #print(self.srs_g1)
        #print(self.srs_g2)
        
    # Commit to a polynomial using the SRS.  Returns a point in G1.  The commitment is a linear combination of the SRS points weighted by the coefficients of the polynomial.  If the coefficient is zero, we skip that term.  We use the multiply function from py_ecc.bn128 to compute the scalar multiplication of the SRS point with the coefficient.  We use the add function from py_ecc.bn128 to add the commitments for each term.  The commitment is returned in G1.  If the polynomial has degree greater than max_degree, we raise a ValueError. 
    def commit(self, poly: Poly):
        coeffs = poly.coefficients()       
        commitment = None
        # Compute the commitment to the polynomial using the SRS
        # Reverse the coefficients as they are stored from highest degree to lowest in Poly object. 
        for coeff, s in zip(coeffs[::-1], self.srs_g1):
            if coeff == 0:
                continue
            term_commitment = multiply(s, int(coeff))
            commitment = add(commitment, term_commitment) if commitment else term_commitment
        # Return the commitment in G1
        # srs = [[1]_1, [s]_1, ..., [s^n]_1]
        # [f(s)]_1 = a_0*[1]_1 + a_1*[s]_1 + ... + a_n*[s^n]_1
        return commitment
    
    def open(self, poly:Poly, z_point):
        f_z = poly(z_point)
        numberator = poly - field(f_z)        
        denominator = Poly([1, -z_point], field=field)
        quotient = numberator //  denominator
        proof = self.commit(quotient)
        return f_z, proof
    
    def verify(self, commitment, z_point, poly:Poly):
        # q(s) = [f(s) - f(z)] / (s - z) in G1
        # [f(s)]_1 - [f(z)]_1 = [q(s)]_1 * (s - z)
        #e([f(s)]_1 - [f(z)]_1, G2) == e([q(s)]_1, [s]_2 - [z]_2)
        f_z, proof = self.open(poly, z_point)
        f_z_g1 = self.commit(Poly([f_z], field=field))        
        s = self.srs
        lhs = add(commitment, neg(f_z_g1))
        lhs_pairing = pairing(self.srs_g2[0], lhs)
        rhs_pairing = pairing(
            add(
                self.srs_g2[1],
                multiply(neg(self.srs_g2[0]), int(z_point))
            ),
            proof
        )
        # Debugging line to see the pairings for comparison. Can be removed in production code.
        #print(f"lhs_pairing: {lhs_pairing}") 
        #print(f"rhs_pairing: {rhs_pairing}") 
        return lhs_pairing == rhs_pairing

In [173]:
kzg = KZGCommitment(3, field)
poly = Poly([1, 2, 3, 0], field=field) # f(x) = x^3 + 2x^2 + 3x + 0
print(poly.coefficients())

[1 2 3 0]


In [174]:
commitment = kzg.commit(poly)
print(commitment)

(9953335635914614939829934415496832188241572235654832090561615548078357654700, 17206522999725458237328610884307598504359551828740947891776895448968538333577)


In [175]:
z_point = field.Random(1)[0]
f_z, proof = kzg.open(poly, z_point)
print(f"f(z) = {f_z}")
print(proof)

f(z) = 15660262963832404716596791646158523948812039572817471521199125877306333397077
(7078526594979175394119242622319255889785551335103852469310179285143485300647, 7205735234049035568519808979835918775081647893054951404282672896406027511544)


In [176]:
kzg.verify(commitment, z_point, poly)

lhs_pairing: (17940726559316152136534235094918619837659637812623765132091811630288608823725, 14532728522988339825372655117465070763600287613277451252610271669628604621895, 21750323611109516835134662252215185355828107691959271561310235113325864279674, 8993021742618597535838162355330249368881480955501870426602755490251716064087, 5923848068248490578184152416742075875201949202674107976583872588642462592295, 5351716301210347756950399198621338277949444063120802284132636617406280799565, 18753758207359458361106877844129195357729447992527245916269151106934417808464, 1460106484377835192317078647277700548086969733587011786488201252088323141894, 462606270038137723191687166701332657554580472112838359195529429572919915087, 8014420729298777074246438256114908870692403611713227623431564701703189355099, 1831253566955229352198209913268943194401786231369425401843794404050798253496, 1477998165994918686176343221846286474189090168578955737444751540303209919772)
rhs_pairing: (179407265593161521365342350949186

True

In [183]:
import json
from collections import defaultdict

with open("r1cs.json") as f:
    r1cs_data = json.load(f)
    
with open("witness.json") as f:
    witness_data = json.load(f)

In [234]:
class PlonkIOP:
    def __init__(self, r1cs_json_path, witness_json_path, field):
        self.field = field
        with open(r1cs_json_path) as f:
            r1cs_data = json.load(f)

        with open(witness_json_path) as f:
            witness_data = json.load(f)

        self.n_constraints = r1cs_data["nConstraints"]
        self.nVars = r1cs_data['nVars']
        self.constraints = r1cs_data['constraints']
        self.nOutputs = r1cs_data['nOutputs']
        self.nPubInputs = r1cs_data['nPubInputs']

        self.witness = [self.field(int(x, 16)) for x in witness_data['witness']]

        # 중요: 필드 원소로 초기화
        self.QL = [self.field(0)] * self.n_constraints
        self.QR = [self.field(0)] * self.n_constraints
        self.QO = [self.field(0)] * self.n_constraints
        self.QM = [self.field(0)] * self.n_constraints
        self.PI = [self.field(0)] * self.n_constraints

        for i in range(min(self.nPubInputs, self.n_constraints)):
            self.PI[i] = self.witness[self.nOutputs + i + 1]

        for i, c in enumerate(self.constraints):
            A_terms = c[0]
            B_terms = c[1]
            C_terms = c[2]

            if A_terms:
                self.QL[i] = sum([self.field(int(v)) for v in A_terms.values()], start=self.field(0))
            if B_terms:
                self.QR[i] = sum([self.field(int(v)) for v in B_terms.values()], start=self.field(0))
            if C_terms:
                self.QO[i] = -sum([self.field(int(v)) for v in C_terms.values()], start=self.field(0))

    def evaluate_gate(self, i, a, b, c):
        return (
            self.QL[i] * a +
            self.QR[i] * b +
            self.QM[i] * a * b +
            self.QO[i] * c +
            self.PI[i]
        )

    def verify(self):
        for i, (A_term, B_term, C_term) in enumerate(self.constraints):
            a_val = self.field(0)
            b_val = self.field(0)
            c_val = self.field(0)

            if A_term:
                a_wire = int(next(iter(A_term.keys())))
                a_val = self.witness[a_wire]
            if B_term:
                b_wire = int(next(iter(B_term.keys())))
                b_val = self.witness[b_wire]
            if C_term:
                c_wire = int(next(iter(C_term.keys())))
                c_val = self.witness[c_wire]

            eval_result = self.evaluate_gate(i, a_val, b_val, c_val)
            if eval_result != self.field(0):
                print(f"❌ Constraint {i} failed: result={eval_result}, a={a_val}, b={b_val}, c={c_val}")
                return False
        print("✅ All constraints satisfied.")
        return True


In [235]:
r1cs_json_path = 'r1cs.json'
witness_json_path = 'witness.json'

plonkIoP = PlonkIOP(r1cs_json_path, witness_json_path, field)
plonkIoP.verify()

❌ Constraint 0 failed: result=8, a=3, b=2, c=6


False

In [212]:
a = {1:1}
a.keys()[0]

TypeError: 'dict_keys' object is not subscriptable